# Workbook \#3 - Prepare a packet of pris tickets for a tiff stack
#### Steps:
1. take a tiff-stack as the input, and decompose it into a collection if single tiff images, which is going to be handled independently.
2. Prepare/choose a ticket type.
3. Configure the ticket and save out

In [1]:
import sys
sys.path.append("/u/home/x/xiyuyi/bin")
sys.path.append("../PyPRIS")
sys.path.append('/Users/yi10/PyPRIS')
from PyPRIS import *
import copy
import os

### Decompose tiff stack

In [18]:
batchN=15
path_head = '/Users/yi10/Desktop/Research/Projects/P-SPT-PRIS-RAS/simulations/Simu3_validation_of_fittingPerformance_static2D_varDensity'
tiff_stack_file = path_head + '/chara_Batch'+str(batchN)+'_noisy_bg20_timeBin1.tif'
tiff_stack_output_path = path_head + '/prep_for_PRIS/Pack_chara_Batch'+str(batchN)+'_noisy_bg20_timeBin1'
tiff_stack_ticket_folder = path_head + '/prep_for_PRIS/tickets_chara_Batch'+str(batchN)+'_noisy_bg20_timeBin1'
try:
    os.mkdir(tiff_stack_output_path)
except:
    pass
# make a folder and make output as frame1.tif  to frameN.tif
stack=io.imread(tiff_stack_file);
for i in np.arange(len(stack)):
    io.imsave(tiff_stack_output_path+'/frame'+str(i)+'.tif', stack[i])

### prepare a ticket
we are going to use SinglePlane ticket here.

In [27]:
ticket = SinglePlaneTicket()

"where to find the data files, for both blur and observation"
paths = []
paths.append(tiff_stack_output_path)
#paths.append('/p/lscratchh/yi10/spt-ras/frames')
ax0_ranges = []
ax0_ranges.append(list([0,1])) # this option with give you 1 slice in PSF model, equivalent to a 2D PSF model.

ticket_folders = []
ticket_folders.append(tiff_stack_ticket_folder)
for frameN in list(np.arange(0,5)):
    print(frameN)
    for datapath, ax0_range, ticket_folder in zip(paths, ax0_ranges, ticket_folders):
        try:
            os.mkdir(ticket_folder)
        except:
            pass
        ticket.datapath = datapath
        ticket.observation_channel_n = 'SinglePlane'
        ticket.plane1_path = "{}/frame{}.tif".format(ticket.datapath,str(frameN))
        ticket.psf_path = "{}/psf.tif".format(ticket.datapath)
        ticket.psf_norm_factor = 1 # The PSF normalization factor is a number that 
                                      # normalizes the total energy of a PSF to be 1
                                      # But experimental calibration data, the PSF matrix is often noisy, 
                                      # so an estimate would be enough.
                    # when the background is too strong, we change this to a smaller number.. forces the scale of the signal measures to be higher

        "specification of the psf stack"
        ticket.psfz0 = 101  # the count of the center plane, define it as z=0 in the psf coordinates system.
        ticket.observer_edge_padding = False # switch to True when PSF has high bg component.

        "configure the initial candidate pool of this pris ticket"
        ticket.init_candidates_intervals = list([1, 1, 1])
        
        ticket.init_ax0_range = ax0_range
        ticket.init_ax1_range = list([0, 128])
        ticket.init_ax2_range = list([0, 128])
#        ticket.init_ax1_range = list([0, 128])
#        ticket.init_ax2_range = list([0, 128])

        "debug configurations"
        ticket.observer_debugger = False
        ticket.tobserver_edge_padding = True

        "output settings"
        ticket.ticket_folder = ticket_folder

        "linbreg configurations"
        ticket.linbreg_alpha = LinBreg("X")
        ticket.linbreg_alpha.debug = True
        ticket.linbreg_alpha.deep_debug = False

        "ticket.linbreg_alpha.mu = 1000000000"  # move to loop
        "ticket.linbreg_alpha.alpha = 1e-11"  # move to loop
        ticket.linbreg_alpha.maxit = 3000
        ticket.linbreg_alpha.it_check_rem = 0
        ticket.linbreg_alpha.debug_it_int = 500
        ticket.linbreg_alpha.kick.ints = ticket.linbreg_alpha.maxit # dont kick when density is high.
        ticket.linbreg_alpha.kick.eval_ints = 2000
        ticket.linbreg_alpha.kick.flag = True
        ticket.linbreg_alpha.kick.thres = 0.1
        ticket.linbreg_alpha.save_obj_int = 2000
        ticket.linbreg_alpha.flag_bg_allow_negative = False # force background to not be negative
        ticket.linbreg_alpha.save = True
        ticket.expansion = False
        ticket.linbreg_alpha.PyPRIS_iter = 0
        "ticket.linbreg_alpha.PyPRIS_name = ticket.name"  # moved to loop
        ticket.linbreg_alpha.path_0 = '.'
        "ticket.bg_scaling_coef = 1.5 "  # moved to loop
        ticket.linbreg_alpha.stopping_loghistpercdelres_thres = -16

        "others"
        ticket.PRIS_iter_end = 5
        try:
            if not os.path.exists("../{}".format(ticket.ticket_folder)):
                os.mkdir("../{}".format(ticket.ticket_folder))
        except OSError:
            pass

        for bgSCF in list([10]): # the bgSCF is background scaling factor. The scales the amplitude of desired background component.
            for mu in list([1e6]):
                for alpha in list([1e-6]):
                    ticket_new = copy.deepcopy(ticket)
                    #ticket_new.name = "bgSCF" + str(bgSCF) + "_mu" + str("%1.1e" % mu) + "_alpha" + str("%1.1e" % alpha)
                    ticket_new.name="frame"+str(frameN)
                    ticket_new.bg_scaling_coef = copy.deepcopy(bgSCF)
                    ticket_new.linbreg_alpha.PyPRIS_name = ticket_new.name
                    ticket_new.linbreg_alpha.mu = mu
                    ticket_new.linbreg_alpha.alpha = alpha
                    ticket_new.linbreg_alpha.auto_mu = True
                    ticket_new.linbreg_alpha.auto_mu_fold = 100;
                    ticket_new.linbreg_alpha.auto_bg = True
                    ticket_new.linbreg_alpha.auto_bg_fold = 2  # increase number to make the bg compoentn more competitive than random background 'non-specific-emitters'.
                                                                # use a larger number when you think the background component in the calibrated PSF matrix
                        # is significantly smaller than tht in the data. 
                        # you may consider using a number smaller than 1 when you thin the background component in PSF calibration is big or comparable.
                    try:
                        if not os.path.exists("{}/{}".format(ticket_new.ticket_folder, ticket_new.name)):
                            os.mkdir("{}/{}".format(ticket_new.ticket_folder, ticket_new.name))
                    except OSError:
                        pass
                    with open("{}/{}/Go.pris_ticket".format(ticket_new.ticket_folder, ticket_new.name), "wb") as f:
                        pickle.dump(ticket_new, f, pickle.HIGHEST_PROTOCOL)


0
1
2
3
4


# Now run pris from terminal. 
use pris_moti

seems like bg1 mu100 worked! increase bg to see if it converges a bit faster
mu100, bg2 no big differences



## Check Result

refer this workbook below for clustering and classification to convert CS result into a list of localization fittings, and inspection of a single CS result.

`~/workbooks/WB#1 - Inspect one fitting result from the compressive sensing solver.ipynb`

Refer to this workbook below for visualizations of pris result from pris order 0 to 4

`~/workbooks/WB#2 - Inspect fitting results from pris0 to pris4.ipynb`